In [1]:
! python3 -m ipykernel install --user --name=main-venv

Installed kernelspec main-venv in /home/roman/.local/share/jupyter/kernels/main-venv


In [2]:
import os

import json

from tqdm import tqdm

import pandas as pd
import numpy as np

from pydantic import BaseModel, Field

from typing import List

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.exceptions import OutputParserException

from langchain_gigachat.chat_models import GigaChat

from parser.static_parsers import build_kupus_parser
from parser.static_parsers import build_hranidengi_parser
from parser.static_parsers import build_findozor_parser
from parser.static_parsers import build_finforum_parser

In [3]:
DATA_DIR = "data"

# ✅Build Giga based solution

In [4]:
class GigaOutput(BaseModel):

    topic: str = Field(description='''
        Описание темы на форуме.
    ''')
    menace_status: str = Field(description='''
        Ответ от Гигачата на вопрос, является ли данная
        тема на форуме потенциально мошеннической.
    '''),
    comment: str = Field(description='''
        Небольшой комментарий от Гигачата,
        почему он посчитал тему потенциально мошеннической.
    ''')

class GigaMapper:

    def __init__(self, token: str, prompt: str, model: str = 'GigaChat-2'):

        self.model = GigaChat(
            credentials=token,
            verify_ssl_certs=False,
            model=model
        ).with_structured_output(GigaOutput, method="json_mode")

        self.messages = [
            SystemMessage(content=prompt)
        ]

    def inference(self, topics: List[str]):

        res = []
        
        for topic in tqdm(topics):

            try:
                answer = self.model.invoke(self.messages + [HumanMessage(topic)])
            except OutputParserException:
                answer = {'topic': topic, 'menace_status': 'да', 'comment': 'чувствительная тема'}
                
            res.append(dict(answer))
        
        return res

In [5]:
with open('gc-token.txt', 'r') as file:
    TOKEN = file.read()

system_prompt = '''
    Ты - помошник по борьбе с онлайн мошенничеством в банке.
    Тебе дают заголовок темы на одном из онлайн банковских форумов.
    Твоя задача - определить, является ли тема потенциально мошеннической - 
    если да, ответь "да", иначе - "нет".
    Также дай небольшой комментарий, почему ты так решил.

    Проанализируй тему и верни ответ в формате JSON со структурой:
        {
          "topic": "оригинальное название темы"
          "menace_status": "да/нет",
          "comment": "твой короткомментарий"
        }

    Запрещено:
        - Добавлять посторонний текст
        - Менять структуру JSON
'''

giga = GigaMapper(TOKEN, system_prompt)

# ✅Parse and map Kupus.ru topics

## Parse Kupus.ru

In [5]:
kupus_parser = build_kupus_parser()
kupus_topics = kupus_parser.parse_topics(
    "https://kupus.ru/forum/17-kredity/"
)

## Giga inference

In [6]:
kupus_topics_df = pd.DataFrame(giga.inference(kupus_topics.keys()))

100%|███████████████████████████████████████████| 20/20 [00:10<00:00,  1.92it/s]


In [10]:
kupus_topics_df.to_csv(os.path.join(DATA_DIR, 'kupus_mapping_credits.csv'))

# ✅Parse and map Hranidengi.com topics

## Parse Hranidengi.com

In [6]:
hranidengi_parser = build_hranidengi_parser()
hranidengi_topics = hranidengi_parser.parse_topics(
    "https://hranidengi.com/forums/kreditnaja-istorija/"
)

## Giga inference

In [7]:
hranidengi_topics_df = pd.DataFrame(giga.inference(hranidengi_topics.keys()))

100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.10it/s]


In [9]:
hranidengi_topics_df.to_csv(os.path.join(DATA_DIR, 'hranidengi_mapping_credit_story.csv'))

# ✅Parse and map Findozor.net topics

## Parse Findozor.net

In [6]:
findozor_parser = build_findozor_parser()
findozor_topics = findozor_parser.parse_topics(
    "https://findozor.net/forum/forums/banki2/",
    deep=5
)

## Giga inference

In [7]:
findozor_topics_df = pd.DataFrame(giga.inference(findozor_topics.keys()))

100%|███████████████████████████████████████████| 53/53 [00:27<00:00,  1.96it/s]


In [9]:
findozor_topics_df.to_csv(os.path.join(DATA_DIR, 'findozor_mapping_credit_story.csv'))

# ✅Parse and map Findozor.net topics

## Parse Findozor.net

In [6]:
findozor_parser = build_findozor_parser()
findozor_topics = findozor_parser.parse_topics(
    "https://findozor.net/forum/forums/banki2/",
    deep=5
)

## Giga inference

In [7]:
findozor_topics_df = pd.DataFrame(giga.inference(findozor_topics.keys()))

100%|███████████████████████████████████████████| 53/53 [00:27<00:00,  1.96it/s]


In [9]:
findozor_topics_df.to_csv(os.path.join(DATA_DIR, 'findozor_mapping_credit_story.csv'))

# ✅Parse and map Finforum.ru topics

## Parse Finforum.ru

In [6]:
finforum_parser = build_finforum_parser()
finforum_topics = finforum_parser.parse_topics(
    "https://finforums.ru/forums/kreditnyye-istorii.46/"
)

## Giga inference

In [8]:
finforum_topics_df = pd.DataFrame(giga.inference(finforum_topics.keys()))

100%|█████████████████████████████████████████████| 6/6 [00:03<00:00,  1.81it/s]


In [9]:
finforum_topics_df.to_csv(os.path.join(DATA_DIR, 'finforum_mapping_credit_story.csv'))